In [ ]:
from mmseg.registry import MODELS
import torch
import torch.nn as nn
from mmcv.cnn import ConvModule

from mmseg.registry import MODELS
from mmseg.models.decode_heads.decode_head import BaseDecodeHead


from mmseg.models.utils import resize,UpConvBlock, Upsample
from mmseg.models.decode_heads.psp_head import PPM

import torch.utils.checkpoint as cp
from mmcv.cnn import ConvModule, build_activation_layer, build_norm_layer
from mmengine.model import BaseModule
from mmengine.utils.dl_utils.parrots_wrapper import _BatchNorm

class BasicConvBlock(nn.Module):
    """Basic convolutional block for UNet.

    This module consists of several plain convolutional layers.

    Args:
        in_channels (int): Number of input channels.
        out_channels (int): Number of output channels.
        num_convs (int): Number of convolutional layers. Default: 2.
        stride (int): Whether use stride convolution to downsample
            the input feature map. If stride=2, it only uses stride convolution
            in the first convolutional layer to downsample the input feature
            map. Options are 1 or 2. Default: 1.
        dilation (int): Whether use dilated convolution to expand the
            receptive field. Set dilation rate of each convolutional layer and
            the dilation rate of the first convolutional layer is always 1.
            Default: 1.
        with_cp (bool): Use checkpoint or not. Using checkpoint will save some
            memory while slowing down the training speed. Default: False.
        conv_cfg (dict | None): Config dict for convolution layer.
            Default: None.
        norm_cfg (dict | None): Config dict for normalization layer.
            Default: dict(type='BN').
        act_cfg (dict | None): Config dict for activation layer in ConvModule.
            Default: dict(type='ReLU').
        dcn (bool): Use deformable convolution in convolutional layer or not.
            Default: None.
        plugins (dict): plugins for convolutional layers. Default: None.
    """

    def __init__(self,
                 in_channels,
                 out_channels,
                 num_convs=2,
                 stride=1,
                 dilation=1,
                 with_cp=False,
                 conv_cfg=None,
                 norm_cfg=dict(type='BN'),
                 act_cfg=dict(type='ReLU'),
                 dcn=None,
                 plugins=None):
        super().__init__()
        assert dcn is None, 'Not implemented yet.'
        assert plugins is None, 'Not implemented yet.'

        self.with_cp = with_cp
        convs = []
        for i in range(num_convs):
            convs.append(
                ConvModule(
                    in_channels=in_channels if i == 0 else out_channels,
                    out_channels=out_channels,
                    kernel_size=3,
                    stride=stride if i == 0 else 1,
                    dilation=1 if i == 0 else dilation,
                    padding=1 if i == 0 else dilation,
                    conv_cfg=conv_cfg,
                    norm_cfg=norm_cfg,
                    act_cfg=act_cfg))

        self.convs = nn.Sequential(*convs)

    def forward(self, x):
        """Forward function."""

        if self.with_cp and x.requires_grad:
            out = cp.checkpoint(self.convs, x)
        else:
            out = self.convs(x)
        return out


@MODELS.register_module()
class UnetDecoder(BaseDecodeHead):
    """Unified Perceptual Parsing for Scene Understanding.

    This head is the implementation of `UPerNet
    <https://arxiv.org/abs/1807.10221>`_.

    Args:
        pool_scales (tuple[int]): Pooling scales used in Pooling Pyramid
            Module applied on the last feature. Default: (1, 2, 3, 6).
    """

    def __init__(self, num_stages = 3, **kwargs):
        super().__init__(input_transform='multiple_select', **kwargs)
        # PSP Module
        upsample_cfg=dict(type='InterpConv')
        self.decoder = nn.ModuleList()
        self.num_stages = num_stages
#         self.encoder = []
#         self.encoder.append(BasicConvBlock(
#                     in_channels=self.in_channels[-1],
#                     out_channels=self.in_channels[-1] ** 2,
#                     num_convs=2,
#                     stride=1,
#                     dilation=1,
#                     with_cp=False,
#                     conv_cfg=None,
#                     norm_cfg=dict(type='BN'),
#                     act_cfg=dict(type='ReLU'),
#                     dcn=None,
#                     plugins=None))
        for i in range(num_stages - 1):
            #print(self.in_channels[num_stages - i-1])
            #print(self.in_channels[num_stages -i-2])
            self.decoder.append(
                    UpConvBlock(
                        conv_block=BasicConvBlock,
                        in_channels=self.in_channels[num_stages - i-1],
                        skip_channels=self.in_channels[num_stages -i-2],
                        out_channels=self.in_channels[num_stages -i-2],
                        num_convs=2,
                        stride=1,
                        dilation=1,
                        with_cp=False,
                        conv_cfg=None,
                        norm_cfg=dict(type='BN'),
                        act_cfg=dict(type='ReLU'),
                        upsample_cfg=upsample_cfg,
                        dcn=None,
                        plugins=None))



    def _forward_feature(self, inputs):
        """Forward function for feature maps before classifying each pixel with
        ``self.cls_seg`` fc.

        Args:
            inputs (list[Tensor]): List of multi-level img features.

        Returns:
            feats (Tensor): A tensor of shape (batch_size, self.channels,
                H, W) which is feature map for last layer of decoder head.
        """
        inputs = self._transform_inputs(inputs)

#         print(len(inputs))
#         for inp in inputs:
#             print(inp.shape)
        x = inputs[-1]
        for i in range(self.num_stages - 1):
            x = self.decoder[i](inputs[-2-i],x)
            #print(x.shape)
        #bottleneck = self.bottleneck(inputs[-1])
        #print(bottleneck.shape)
        return x

    def forward(self, inputs):
        """Forward function."""
        output = self._forward_feature(inputs)
        output = self.cls_seg(output)
        return output

In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from mmwrapper.src.api import get_runner

C:\Users\philmarq\miniconda3\envs\mmtest\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
runner = get_runner(r"C:\Users\philmarq\source\repos\MMWrapperRepo\mmwrapper\src\configs\object_detection\object_detection.yaml")
#runner = get_runner(r"C:\Users\philmarq\source\repos\MMWrapperRepo\mmwrapper\src\configs\segmentation\segmentation.yaml")
#runner = get_runner(r"C:\Users\philmarq\source\repos\MMWrapperRepo\mmwrapper\src\configs\instance_segmentation\instance_seg.yaml")

In [3]:
runner.train()

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection/v3.0/dino/dino-4scale_r50_8xb2-12e_coco/dino-4scale_r50_8xb2-12e_coco_20221202_182705-55b2bba2.pth
The model and loaded state dict do not match exactly

size mismatch for backbone.conv1.weight: copying a param with shape torch.Size([64, 3, 7, 7]) from checkpoint, the shape in current model is torch.Size([64, 2, 7, 7]).
size mismatch for bbox_head.cls_branches.0.weight: copying a param with shape torch.Size([80, 256]) from checkpoint, the shape in current model is torch.Size([2, 256]).
size mismatch for bbox_head.cls_branches.0.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([2]).
size mismatch for

C:\Users\philmarq\miniconda3\envs\mmtest\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


IndexError: list index out of range

# Instance seg inference

In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#import pydensecrf.densecrf as dcrf

from mmdet.apis import init_detector, inference_detector

import numpy as np
import cv2



config_file = r"C:\Users\philmarq\source\repos\MMWrapperRepo\megdata\swin_s.py"
checkpoint_file = r"C:\Users\philmarq\source\repos\MMWrapperRepo\megdata\best_coco_bbox_mAP_epoch_55.pth"

device = 'cuda:0'

model = init_detector(config_file, checkpoint_file, device=device)

# Load image


colors = [(255,0,0), (125,0,0), (0,255,0), (50, 255,0), (0,25,255), (0, 125, 0)]
root = r"C:\HSA\HSA-Data\projects\2cc375cc-4067-40f6-a23d-387ec7422611\dataset\images"
for file in os.listdir(root):
    img = os.path.join(root, file)
    result = inference_detector(model, img)
    
    

    color = (255,0,0)
    image = cv2.imread(img)
    #image[image>0] = 0
    masks = result.pred_instances.masks.cpu().numpy()
    scores = result.pred_instances.scores.cpu().numpy()
    labels = result.pred_instances.labels.cpu().numpy()
    bboxes = result.pred_instances.bboxes.cpu().numpy()
    thresh = 0.75
    
    for mask, score,label, bbox in zip(masks, scores,  labels, bboxes):
        if score > thresh:
            if label == 0:  # For label == 0, draw contours
                mask = mask.astype(np.uint8)
                contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                #contours = connect_closest_contours(contours, image, angle_tolerance = 5, fixed_threshold=50, pixel_threshold = 3)
                #color  = np.random.rand(3) * 255
                cv2.drawContours(image, contours, -1, color, 1)
                x1, y1, x2, y2 = bbox.astype(np.int32)
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 1)
            else:  # For other labels, draw bounding boxes
                color = colors[label]
                x1, y1, x2, y2 = bbox.astype(np.int32)
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 1)
    
    cv2.imwrite(os.path.join("outputall", file), image)

In [ ]:
import torch

torch.cuda.empty_cache()

In [ ]:
import tifffile

In [ ]:
img = tifffile.imread(r"C:\Users\philmarq\Downloads\dataset_seg_2ch_256_updated\dataset_seg_2ch_256_updated\valid\images\T20_CD3_CD8_whole_scene-0_base_roi-0_x-768_y-960.tif")

In [ ]:
img.shape